# DDACS Dataset Tutorial

This tutorial demonstrates how to work with the Deep Drawing and Cutting Simulations (DDACS) Dataset.

## Dataset Overview

The DDACS dataset contains FEM simulation results of deep drawing processes with:

**Process Parameters:**
- **FC**: Friction coefficient
- **MAT**: Material properties 
- **STHK**: Sheet thickness [mm]
- **BF**: Blank holder force [N]

**Geometry Types:**
- **R**: Rectangular geometry + radius
- **V**: Concave geometry + radius  
- **X**: Convex geometry + radius

**Components:**
- **Blank**: Sheet metal workpiece being formed
- **Die, Punch, Binder**: Forming tools

**Operations:**
- **OP10**: Main forming process (multiple timesteps)
- **OP20**: Springback analysis after tool removal

## Installation

```bash
pip install "git+https://github.com/BaumSebastian/Deep-Drawing-and-Cutting-Simulations-Dataset.git[examples]"
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
from pathlib import Path
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from ddacs import DDACSDataset
from ddacs.utils import extract_point_cloud, extract_mesh

## 1. Loading the Dataset

In [ ]:
# Setup data directory
data_dir = Path("./../data")

# Load dataset
dataset = DDACSDataset(data_dir, "h5")
print(f"Loaded {len(dataset)} simulations")

# Get a sample simulation
sim_id, metadata, h5_path = dataset[0]
print(f"Sample simulation: {sim_id}")
print(f"File: {h5_path.name}")
print(f"Metadata: {metadata}")

## 2. Understanding the Metadata

The metadata contains the process and geometry parameters that define each simulation.

In [ ]:
# Load and analyze metadata
metadata_file = data_dir / "metadata.csv"
if metadata_file.exists():
    df = pd.read_csv(metadata_file)
    print(f"Metadata structure:")
    print(f"  Total simulations: {len(df)}")
    print(f"  Columns: {list(df.columns)}")
    
    # Show parameter meanings and ranges
    print(f"\nProcess Parameters (physical values):")
    
    # Note: Update these when you have the actual unnormalized data
    param_info = {
        'FC': 'Friction Coefficient [dimensionless]',
        'MAT': 'Material Properties [various units]', 
        'STHK': 'Sheet Thickness [mm]',
        'BF': 'Blank Holder Force [N]'
    }
    
    for param, description in param_info.items():
        if param in df.columns:
            values = df[param]
            print(f"  {param}: {description}")
            print(f"    Range: {values.min():.3f} to {values.max():.3f}")
            print(f"    Mean: {values.mean():.3f}")
    
    # Show geometry information when available
    if 'GEOMETRY' in df.columns:
        print(f"\nGeometry Types:")
        geom_counts = df['GEOMETRY'].value_counts()
        for geom_type, count in geom_counts.items():
            geom_name = {'R': 'Rectangular', 'V': 'Concave', 'X': 'Convex'}.get(geom_type, geom_type)
            print(f"  {geom_type}: {geom_name} ({count} simulations)")
    
    if 'RADIUS' in df.columns:
        radius_values = df['RADIUS']
        print(f"\nRadius Parameter:")
        print(f"  Range: {radius_values.min():.1f} to {radius_values.max():.1f} mm")
        print(f"  Mean: {radius_values.mean():.1f} mm")
    
    print(f"\nSample simulations:")
    print(df.head())

## 3. Simulation Data Structure

In [ ]:
# Explore the HDF5 simulation structure
with h5py.File(h5_path, 'r') as f:
    print("Operations:", list(f.keys()))
    
    # OP10: Forming operation
    print("\nOP10 - Forming Operation:")
    op10 = f['OP10']
    print("  Components:", list(op10.keys()))
    
    # Analyze each component
    print("\nComponent Details:")
    for component in ['blank', 'die', 'binder', 'punch']:
        if component in op10:
            comp = op10[component]
            nodes = comp['node_coordinates'].shape[0]
            
            if 'node_displacement' in comp:
                disp_shape = comp['node_displacement'].shape
                timesteps = disp_shape[0] if len(disp_shape) == 3 else 1
                print(f"  {component}: {nodes} nodes, {timesteps} timesteps")
            else:
                print(f"  {component}: {nodes} nodes, no displacement")
    
    # OP20: Springback operation (if exists)
    if 'OP20' in f:
        print("\nOP20 - Springback Operation:")
        op20 = f['OP20']
        print("  Components:", list(op20.keys()))
        if 'blank' in op20:
            springback_shape = op20['blank']['node_displacement'].shape
            print(f"  Springback timesteps: {springback_shape[0]}")

## 4. Point Cloud Visualization

In [ ]:
# Extract point cloud for the blank (workpiece)
blank_coords = extract_point_cloud(h5_path, 'blank', timestep=0)
print(f"Blank: {blank_coords.shape[0]} nodes")

# Visualize blank point cloud
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(blank_coords[:, 0], blank_coords[:, 1], blank_coords[:, 2], 
          c='red', s=0.5, alpha=0.6)
ax.set_title(f'Blank (Workpiece) - Simulation {sim_id}')
ax.set_xlabel('X [mm]')
ax.set_ylabel('Y [mm]')
ax.set_zlabel('Z [mm]')
plt.show()

In [ ]:
# Visualize complete forming setup
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Component colors following manufacturing convention
component_info = {
    'blank': {'color': 'red', 'name': 'Blank (Workpiece)'},
    'die': {'color': 'blue', 'name': 'Die (Lower Tool)'},
    'punch': {'color': 'green', 'name': 'Punch (Upper Tool)'},
    'binder': {'color': 'orange', 'name': 'Binder (Clamp)'}
}

for component, info in component_info.items():
    try:
        coords = extract_point_cloud(h5_path, component, timestep=0)
        ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2],
                  c=info['color'], label=info['name'], s=0.5, alpha=0.7)
        print(f"{component}: {coords.shape[0]} nodes")
    except Exception as e:
        print(f"Could not load {component}: {e}")

ax.set_title(f'Deep Drawing Setup - Simulation {sim_id}')
ax.set_xlabel('X [mm]')
ax.set_ylabel('Y [mm]')
ax.set_zlabel('Z [mm]')
ax.legend()
plt.show()

## 5. Mesh Visualization

In [ ]:
# Show forming process: initial vs final state
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), subplot_kw={'projection': '3d'})

# Initial state
vertices, triangles = extract_mesh(h5_path, 'blank', timestep=0)
faces = vertices[triangles]
mesh = Poly3DCollection(faces, alpha=0.8, facecolor='lightblue', 
                       edgecolor='blue', linewidth=0.1)
ax1.add_collection3d(mesh)
ax1.set_title('Initial State (t=0)')

# Final formed state
vertices, triangles = extract_mesh(h5_path, 'blank', timestep=-1)
faces = vertices[triangles]
mesh = Poly3DCollection(faces, alpha=0.8, facecolor='orange', 
                       edgecolor='red', linewidth=0.1)
ax2.add_collection3d(mesh)
ax2.set_title('Final Formed State')

# Set equal scaling
for ax in [ax1, ax2]:
    ax.set_xlabel('X [mm]')
    ax.set_ylabel('Y [mm]')
    ax.set_zlabel('Z [mm]')
    ax.set_xlim(vertices[:, 0].min(), vertices[:, 0].max())
    ax.set_ylim(vertices[:, 1].min(), vertices[:, 1].max())
    ax.set_zlim(vertices[:, 2].min(), vertices[:, 2].max())

plt.suptitle(f'Blank Forming Process - Simulation {sim_id}', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Mesh: {vertices.shape[0]} vertices, {triangles.shape[0]} triangular faces")

In [ ]:
# Complete forming setup with all tools at forming timestep
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Use timestep 1 to show forming in progress
forming_timestep = 1

for component, info in component_info.items():
    try:
        vertices, triangles = extract_mesh(h5_path, component, timestep=forming_timestep)
        faces = vertices[triangles]
        
        # Create mesh with transparency
        alpha = 0.9 if component == 'blank' else 0.7
        mesh = Poly3DCollection(faces, alpha=alpha, facecolor=info['color'],
                               edgecolor='black', linewidth=0.02)
        ax.add_collection3d(mesh)
        
        print(f"{component}: {vertices.shape[0]} vertices, {triangles.shape[0]} faces")
    except Exception as e:
        print(f"Could not create mesh for {component}: {e}")

ax.set_title(f'Deep Drawing Process - Simulation {sim_id} (Timestep {forming_timestep})', fontsize=14)
ax.set_xlabel('X [mm]')
ax.set_ylabel('Y [mm]')
ax.set_zlabel('Z [mm]')

# Create legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=info['color'], label=info['name']) 
                  for info in component_info.values()]
ax.legend(handles=legend_elements, loc='upper left')

# Set good viewing angle
ax.view_init(elev=20, azim=45)
plt.tight_layout()
plt.show()

## 6. Process Parameter Analysis

Example of how to analyze simulations based on their process parameters.

In [ ]:
# Example: Compare simulations with different parameters
if metadata_file.exists():
    df = pd.read_csv(metadata_file)
    
    # Find simulations with extreme parameter values
    print("Parameter analysis:")
    
    for param in ['FC', 'MAT', 'STHK', 'BF']:
        if param in df.columns:
            min_sim = df.loc[df[param].idxmin()]
            max_sim = df.loc[df[param].idxmax()]
            
            print(f"\n{param} parameter:")
            print(f"  Minimum: Sim {min_sim['ID']} with {param}={min_sim[param]:.3f}")
            print(f"  Maximum: Sim {max_sim['ID']} with {param}={max_sim[param]:.3f}")
    
    # Geometry type distribution
    if 'GEOMETRY' in df.columns:
        print(f"\nGeometry distribution:")
        geom_counts = df['GEOMETRY'].value_counts()
        for geom_type, count in geom_counts.items():
            percentage = (count / len(df)) * 100
            geom_name = {'R': 'Rectangular', 'V': 'Concave', 'X': 'Convex'}.get(geom_type, geom_type)
            print(f"  {geom_name} ({geom_type}): {count} simulations ({percentage:.1f}%)")

## 7. Usage Reference

### Dataset Access
```python
from ddacs import DDACSDataset, DDACSIterator, iter_ddacs

# PyTorch-compatible dataset
dataset = DDACSDataset(data_dir, "h5")
sim_id, metadata, h5_path = dataset[0]

# Lightweight iteration
iterator = DDACSIterator(data_dir, "h5")
for sim_id, metadata, h5_path in iterator:
    # Process simulation
    pass
```

### Data Extraction
```python
from ddacs.utils import extract_point_cloud, extract_mesh

# Extract coordinates
coords = extract_point_cloud(h5_path, 'blank', timestep=0)

# Extract mesh for visualization
vertices, triangles = extract_mesh(h5_path, 'blank', timestep=-1)
```

### Metadata Structure
- **FC**: Friction coefficient [dimensionless]
- **MAT**: Material properties [various units]
- **STHK**: Sheet thickness [mm]
- **BF**: Blank holder force [N]
- **GEOMETRY**: R (rectangular), V (concave), X (convex)
- **RADIUS**: Geometry radius parameter [mm]

### Operations & Timesteps
- **OP10**: Forming operation (4 timesteps for blank, 3 for tools)
- **OP20**: Springback analysis (2 timesteps, blank only)
- **Timestep 0**: Initial state
- **Timestep -1**: Final state

### Components
- **blank**: Sheet metal workpiece (most data available)
- **die**: Lower forming tool
- **punch**: Upper forming tool  
- **binder**: Edge clamping tool